<a href="https://colab.research.google.com/github/mimilazarova/dd2412_project_fixmatch_and_beyond/blob/main/src/FixMatch_mimi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install tensorflow-addons==0.11.1

     |████████████████████████████████| 1.1MB 11.0MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [22]:
# All imports here
import tensorflow_probability as tfp
import numpy as np
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import os
import tensorflow as tf
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
logging.getLogger('tensorflow').disabled = True
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from tqdm import tqdm, tqdm_notebook
from matplotlib import pyplot as plt
import json
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
import math

'2.3.0'

In [6]:
# this cell is for the wide ResNet
def regularized_padded_conv(*args, **kwargs):
    return tf.keras.layers.Conv2D(*args, **kwargs, padding='same', kernel_regularizer=_regularizer,
                                  kernel_initializer='he_normal', use_bias=False)


def BN_ReLU(x):
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.layers.ReLU()(x)


def shortcut(x, filters, stride, mode):
    if x.shape[-1] == filters:
        return x
    elif mode == 'B':
        return regularized_padded_conv(filters, 1, strides=stride)(x)
    elif mode == 'B_original':
        x = regularized_padded_conv(filters, 1, strides=stride)(x)
        return tf.keras.layers.BatchNormalization()(x)
    elif mode == 'A':
        return tf.pad(tf.keras.layers.MaxPool2D(1, stride)(x) if stride>1 else x,
                      paddings=[(0, 0), (0, 0), (0, 0), (0, filters - x.shape[-1])])
    else:
        raise KeyError("Parameter shortcut_type not recognized!")
    

def original_block(x, filters, stride=1, **kwargs):
    c1 = regularized_padded_conv(filters, 3, strides=stride)(x)
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    c2 = tf.keras.layers.BatchNormalization()(c2)
    
    mode = 'B_original' if _shortcut_type == 'B' else _shortcut_type
    x = shortcut(x, filters, stride, mode=mode)
    return tf.keras.layers.ReLU()(x + c2)
    
    
def preactivation_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
        
    c1 = regularized_padded_conv(filters, 3, strides=stride)(flow)
    if _dropout:
        c1 = tf.keras.layers.Dropout(_dropout)(c1)
        
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c2


def bootleneck_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
         
    c1 = regularized_padded_conv(filters//_bootleneck_width, 1)(flow)
    c2 = regularized_padded_conv(filters//_bootleneck_width, 3, strides=stride)(BN_ReLU(c1))
    c3 = regularized_padded_conv(filters, 1)(BN_ReLU(c2))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c3


def group_of_blocks(x, block_type, num_blocks, filters, stride, block_idx=0):
    global _preact_shortcuts
    preact_block = True if _preact_shortcuts or block_idx == 0 else False
    
    x = block_type(x, filters, stride, preact_block=preact_block)
    for i in range(num_blocks-1):
        x = block_type(x, filters)
    return x


def Resnet(input_shape, n_classes, l2_reg=1e-4, group_sizes=(2, 2, 2), features=(16, 32, 64), strides=(1, 2, 2),
           shortcut_type='B', block_type='preactivated', first_conv={"filters": 16, "kernel_size": 3, "strides": 1},
           dropout=0, cardinality=1, bootleneck_width=4, preact_shortcuts=True):
    
    global _regularizer, _shortcut_type, _preact_projection, _dropout, _cardinality, _bootleneck_width, _preact_shortcuts
    _bootleneck_width = bootleneck_width # used in ResNeXts and bootleneck blocks
    _regularizer = tf.keras.regularizers.l2(l2_reg)
    _shortcut_type = shortcut_type # used in blocks
    _cardinality = cardinality # used in ResNeXts
    _dropout = dropout # used in Wide ResNets
    _preact_shortcuts = preact_shortcuts
    
    block_types = {'preactivated': preactivation_block,
                   'bootleneck': bootleneck_block,
                   'original': original_block}
    
    selected_block = block_types[block_type]
    inputs = tf.keras.layers.Input(shape=input_shape)
    flow = regularized_padded_conv(**first_conv)(inputs)
    
    if block_type == 'original':
        flow = BN_ReLU(flow)
    
    for block_idx, (group_size, feature, stride) in enumerate(zip(group_sizes, features, strides)):
        flow = group_of_blocks(flow,
                               block_type=selected_block,
                               num_blocks=group_size,
                               block_idx=block_idx,
                               filters=feature,
                               stride=stride)
    
    if block_type != 'original':
        flow = BN_ReLU(flow)
    
    flow = tf.keras.layers.GlobalAveragePooling2D()(flow)
    outputs = tf.keras.layers.Dense(n_classes, kernel_regularizer=_regularizer)(flow)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def load_weights_func(model, model_name):
    try: model.load_weights(os.path.join('saved_models', model_name + '.tf'))
    except tf.errors.NotFoundError: print("No weights found for this model!")
    return model


def cifar_wide_resnet(N, K, block_type='preactivated', shortcut_type='B', dropout=0, l2_reg=2.5e-4):
    assert (N-4) % 6 == 0, "N-4 has to be divisible by 6"
    lpb = (N-4) // 6 # layers per block - since N is total number of convolutional layers in Wide ResNet
    model = Resnet(input_shape=(32, 32, 3), n_classes=10, l2_reg=l2_reg, group_sizes=(lpb, lpb, lpb), features=(16*K, 32*K, 64*K),
                   strides=(1, 2, 2), first_conv={"filters": 16, "kernel_size": 3, "strides": 1}, shortcut_type=shortcut_type,
                   block_type=block_type, dropout=dropout, preact_shortcuts=True)
    return model


def WRN_28_2(shortcut_type='B', load_weights=False, dropout=0, l2_reg=2.5e-4):
    model = cifar_wide_resnet(28, 2, 'preactivated', shortcut_type, dropout=dropout, l2_reg=l2_reg)
    if load_weights: model = load_weights_func(model, 'cifar_WRN_28_10')
    return model

In [23]:
# This cell is for CTAugment

class CTAugment:

  def __init__(self, n_classes, decay=0.99, threshold=0.80, depth=2, n_bins=17):
    self.decay = decay
    self.threshold = threshold
    self.depth = depth
    self.n_bins = n_bins
    self.n_classes = n_classes
    self.xforms = []
    # self.bins = [[]]
    # self.weights = [[]]

    self.AUG_DICT = {
        "autocontrast": {"f": self.autocontrast, "weight": [np.ones(self.n_bins)*1.0]},
        "blur": {"f": self.blur, "weight": [np.ones(self.n_bins)*1.0]},
        "brightness": {"f": self.brightness, "weight":[np.ones(self.n_bins)*1.0]},
        "color": {"f": self.color, "weight": [np.ones(self.n_bins)*1.0]},
        "contrast": {"f": self.contrast, "weight": [np.ones(self.n_bins)*1.0]},
        "cutout": {"f": self.cutout, "weight": [np.ones(self.n_bins)*1.0]},
        "equalize": {"f": self.equalize, "weight": [np.ones(self.n_bins)*1.0]},
        "invert": {"f": self.invert, "weight": [np.ones(self.n_bins)*1.0]},
        "identity": {"f": self.identity, "weight": [np.ones(self.n_bins)*1.0]},
        "posterize": {"f": self.posterize, "weight": [np.ones(self.n_bins)*1.0]},
        "rescale": {"f": self.rescale, "weight": [np.ones(self.n_bins)*1.0, np.ones(6)*1.0]},
        "rotate": {"f": self.rotate, "weight": [np.ones(self.n_bins)*1.0]},
        "sharpness": {"f": self.sharpness, "weight": [np.ones(self.n_bins)*1.0]},
        "shear_x": {"f": self.shear_x, "weight": [np.ones(self.n_bins)*1.0]},
        "shear_y": {"f": self.shear_y, "weight": [np.ones(self.n_bins)*1.0]},
        "smooth": {"f": self.smooth, "weight": [np.ones(self.n_bins)*1.0]},
        "solarize": {"f": self.solarize, "weight": [np.ones(self.n_bins)*1.0]},
        "translate_x": {"f": self.translate_x, "weight": [np.ones(self.n_bins)*1.0]},
        "translate_y": {"f": self.translate_y, "weight": [np.ones(self.n_bins)*1.0]}
    }
    self.N = len(self.AUG_DICT.keys())
    self.options = list(self.AUG_DICT.keys())

    self.batch_choices = []
    self.batch_bins = []

  def weight_to_p(self, weight):
        p = weight + (1 - self.decay)  # Avoid to have all zero.
        p = p / p.max()
        p[p < self.threshold] = 0
        return p/np.sum(p)

  def augment(self, x, uniform_bin_sampling=False):
    aug_x = Image.fromarray(np.uint8(x))#255*x))

    choices = [self.options[i] for i in np.random.choice(np.arange(self.N), self.depth, replace=False)]
    bins = []

    for k in range(self.depth):
        choice_key = choices[k]
        
        transformation = self.AUG_DICT[choice_key]["f"]
        # sample bins
        if uniform_bin_sampling:
          p = np.ones(self.N)/self.N
        else:
          w = self.AUG_DICT[choice_key]["weight"][0]
          p = self.weight_to_p(w)
        curr_bins = {}
        curr_bins["bin"] = np.random.choice(np.arange(self.n_bins), p=p)

        if choice_key=="rescale":
          if uniform_bin_sampling:
            p = np.ones(6)/6
          else:
            w = self.AUG_DICT[choice_key]["weight"][1]
            p = self.weight_to_p(w)
          curr_bins["bin2"] = np.random.choice(np.arange(6), p=p)

        aug_x = transformation(aug_x, **curr_bins)
        bins.append(curr_bins)

    return np.array(aug_x), choices, bins

  def augment_batch(self, batch):
    aug_batch = tf.identity(batch)

    #aug_batch = tf.map_fn(aug_batch, self.augment)
    batch_choices = []
    batch_bins = []
    
    if batch.ndim == 3:
      sample, choices, bins = self.augment(sample)
      batch_choices.append(choices)
      batch_bins.append(bins)
    elif batch.ndim == 4:
      for sample in aug_batch:
        sample, choices, bins = self.augment(sample)
        batch_choices.append(choices)
        batch_bins.append(bins)

    return aug_batch, batch_choices, batch_bins

  def update_weights(self, label, pred, choices, bins):
    omega = 1 - (1 / (2*self.n_classes)) * np.sum(tf.math.abs(label - pred))

    for k, choice in enumerate(choices):
      w = self.AUG_DICT[choice]["weight"][0]
      bin = bins[k]["bin"]
      self.AUG_DICT[choice]["weight"][0][bin] = self.decay * w[bin] + (1 - self.decay) * omega
      # print(self.AUG_DICT[choice]["weight"][0])
      if choices[k] == "rescale":
        w = self.AUG_DICT[choice]["weight"][1]
        bin = bins[k]["bin2"]
        self.AUG_DICT[choice]["weight"][1][bin] = self.decay * w[bin] + (1 - self.decay) * omega



  def update_weights_batch(self, labels, preds, choices, bins):
    [self.update_weights(l, p, c, b) for l, p, c, b in zip(labels, preds, choices, bins)]

  def get_param(self, r_min, r_max, bin):
      possible_value = np.linspace(r_min, r_max, self.n_bins)
      return possible_value[bin]

  def autocontrast(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, ImageOps.autocontrast(x), param)
  
  def blur(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, x.filter(ImageFilter.BLUR), param)
  
  def brightness(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Brightness(x).enhance(0.1 + 1.9*param)

  def color(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Color(x).enhance(0.1 + 1.9*param)

  def contrast(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Contrast(x).enhance(0.1 + 1.9*param)

  def cutout(self, x, bin):
    """Taken directlly from FixMatch code"""
    level = self.get_param(0, 0.5, bin)

    size = 1 + int(level * min(x.size) * 0.499)
    img_height, img_width = x.size
    height_loc = np.random.randint(low=0, high=img_height)
    width_loc = np.random.randint(low=0, high=img_width)
    upper_coord = (max(0, height_loc - size // 2), max(0, width_loc - size // 2))
    lower_coord = (min(img_height, height_loc + size // 2), min(img_width, width_loc + size // 2))
    pixels = x.load()  # create the pixel map
    for i in range(upper_coord[0], lower_coord[0]):  # for every col:
        for j in range(upper_coord[1], lower_coord[1]):  # For every row
            pixels[i, j] = (127, 127, 127)  # set the color accordingly
    return x

  def equalize(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, ImageOps.equalize(x), param)

  def invert(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, ImageOps.invert(x), param)
  
  def identity(self, x, bin):
      return x

  def posterize(self, x, bin):
      param = int(self.get_param(0, 8, bin))
      return ImageOps.posterize(x, param)

  def rescale(self, x, bin, bin2):
      param = self.get_param(0.5, 1, bin)
      methods = (Image.ANTIALIAS, Image.BICUBIC, Image.BILINEAR, Image.BOX, Image.HAMMING, Image.NEAREST)
      method = methods[bin2]
      s = x.size
      scale = param*0.25
      crop = (scale * s[0], scale * s[1], s[0] * (1 - scale), s[1] * (1 - scale))
      return x.crop(crop).resize(x.size, method)

  def rotate(self, x, bin):
      param = self.get_param(-45, 45, bin)
      angle = int(np.round((2 * param - 1) * 45))
      return x.rotate(angle)

  def sharpness(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Sharpness(x).enhance(0.1 + 1.9*param)

  def shear_x(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      shear = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, shear, 0, 0, 1, 0))

  def shear_y(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      shear = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, 0, 0, shear, 1, 0))

  def smooth(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, x.filter(ImageFilter.SMOOTH), param)

  def solarize(self, x, bin):
      param = self.get_param(0, 1, bin)
      th = int(param * 255.999)
      return ImageOps.solarize(x, th)

  def translate_x(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      delta = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, 0, delta, 0, 1, 0))

  def translate_y(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      delta = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, 0, 0, 0, 1, delta))


class OurCosineDecay(tf.keras.experimental.CosineDecay):

  def __call__(self, step):
    with ops.name_scope_v2(self.name or "CosineDecay"):
      initial_learning_rate = ops.convert_to_tensor_v2(
          self.initial_learning_rate, name="initial_learning_rate")
      dtype = initial_learning_rate.dtype
      decay_steps = math_ops.cast(self.decay_steps, dtype)

      global_step_recomp = math_ops.cast(step, dtype)
      global_step_recomp = math_ops.minimum(global_step_recomp, decay_steps)
      completed_fraction = global_step_recomp / decay_steps
      cosine_decayed = math_ops.cos(
          constant_op.constant(7/16 * math.pi) * completed_fraction)

      decayed = (1 - self.alpha) * cosine_decayed + self.alpha
      return math_ops.multiply(initial_learning_rate, decayed)

In [136]:

def training(model, ds_l, ds_u, hparams, n_classes, mean=None, std=None,
                   val_interval=2000, log_interval=200, batch_size=128):

    def train_prep(x, y):
        x = tf.cast(x, tf.float32)#/255.
        return x, y

    def valid_prep(x, y):
        x = tf.cast(x, tf.float32)#/255.
        return x, y

    def weak_transformation(x):
      x = tf.image.random_flip_left_right(x)
      max_shift = tf.cast(x.shape[1]*0.125, dtype=tf.dtypes.int32)
      shift = tf.random.uniform([x.shape[0], 2], minval=-max_shift, maxval=max_shift, dtype=tf.dtypes.int32)
      return tfa.image.translate(x, tf.cast(shift, tf.dtypes.float32))
      

    def pseudolabel(class_dist):
        argmax = tf.math.argmax(class_dist, axis=1)
        return tf.one_hot(argmax, class_dist.shape[1])

    def threshold_gate(one_hot, logits, threshold):
        max_probs = tf.math.multiply(one_hot, tf.nn.softmax(logits))
        return tf.cast(max_probs > threshold, max_probs.dtype)# * max_probs

    def split_data_into_arrays(ds):
        images = []
        labels = []
        for ex in ds.take(50000000):
            try:
                images.append(ex[0])
                labels.append(ex[1])
            except:
                break
        return np.stack(images), np.stack(labels)

    
    #@tf.function
    def step(x_l, y_l, x_u, n_classes, training):
        with tf.GradientTape() as tape:

            if y_l.ndim is 1:
              y_l = tf.one_hot(y_l, n_classes)            

            # labeled data
            x_l_weak = weak_transformation(x_l)
            output_l = model(x_l_weak, training)
            
            loss_l = loss_fn(y_l, output_l)

            
            # unlabeled data
            x_u_weak = weak_transformation(x_u)
            output_u_weak = model(x_u_weak, training)  # should this be training or not?
            y_u = pseudolabel(output_u_weak)
            y_u = threshold_gate(y_u, output_u_weak, hparams['tau'])

            x_u_strong, choices, bins = cta.augment_batch(x_u)
            output_u_strong = model(x_u_strong, training)
            cta.update_weights_batch(y_u, output_u_strong, choices, bins)
          
            loss_u = loss_fn(y_u, output_u_strong)

            # tf.print(loss_u)   
            # print(loss_u)   

            #add losses together
            loss = loss_l + hparams['lamda'] * loss_u

        if training:
            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        class_pred = tf.math.argmax(output_l, axis=1)
        tmp = tf.argmax(y_l, axis=1)
        #accuracy(tmp, class_pred)
        labeled_loss(loss_l)
        unlabeled_loss(loss_u)

        labels = np.argmax(y_u, axis=1)
        labels[np.sum(y_u, axis=1) == 0] = -1
        return labels

    schedule = OurCosineDecay(hparams['eta'], hparams['K'])
    optimizer = tf.keras.optimizers.SGD(schedule, momentum=hparams['beta'], nesterov=hparams['nesterov'])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    cta = CTAugment(hparams['cta_classes'], hparams['cta_decay'], hparams['cta_threshold'], hparams['cta_depth'])

    full_x_l, full_y_l = split_data_into_arrays(ds_l)
    full_x_u, _ = split_data_into_arrays(ds_u)
    
    # split into batches
    ds_l = ds_l.map(train_prep).batch(hparams['batch_size']).prefetch(-1)
    ds_u = ds_u.map(train_prep).batch(hparams['batch_size']).prefetch(-1)


    #runid = run_name + '_x' + str(np.random.randint(10000))
    #writer = tf.summary.create_file_writer(logdir + '/' + runid)
    accuracy = tf.metrics.SparseCategoricalAccuracy()
    labeled_loss = tf.metrics.Mean()
    unlabeled_loss = tf.metrics.Mean()
    
    #print(f"RUNID: {runid}")
    #tf.keras.utils.plot_model(model)#, os.path.join('saved_plots', runid + '.png'))    

    training_step = 0
    best_validation_acc = 0
    epochs = 5

    
    for epoch in range(epochs):

        y_u = np.array([])
        for (x_l, y_l), (x_u, _) in tqdm(zip(ds_l, ds_u), desc=f'epoch {epoch+1}/{epochs}',
                         total=val_interval, ncols=100, ascii=True):
            tf.print("step")

            training_step += 1
            y_batch = step(x_l, y_l, x_u, n_classes, training=True)
            y_batch[1] = np.random.randint(0, 9)
            y_u = np.concatenate((y_u, y_batch))
            tf.print(y_batch)

            #idx = np.where(np.count_nonzero(y_u) == n_classes)

            #x_new_l = np.delete(x_u, idx, axis=0)
            #y_new_l = np.delete(y_u, idx, axis=0)

            #x_l = np.concatenate((x_l, x_new_l))
            #y_l = np.concatenate((y_l, y_new_l))
            
            #tf.print(x_new_l)

            if training_step % log_interval == 0:
                #with writer.as_default():
                    loss_l, loss_u, err = labeled_loss.result(), unlabeled_loss.result(), 1-accuracy.result()
                    print(f" loss_l: {loss_l:^6.3f} | loss_u: {loss_u:^6.3f} | err: {err:^6.3f}", end='\r')

                    tf.summary.scalar('train/error_rate', err, training_step)
                    tf.summary.scalar('train/labeled_loss', loss_l, training_step)
                    tf.summary.scalar('train/unlabeled_loss', loss_u, training_step)
                    tf.summary.scalar('train/learnig_rate', optimizer._decayed_lr('float32'), training_step)
                    labeled_loss.reset_states()
                    unlabeled_loss.reset_states()
                    accuracy.reset_states()

        # filter out and move pseudolabeled data from ds_u to ds_l
        #newly_labeled = ds_u.filter(lambda x, y: 1)
        #ds_u = ds_u.filter(lambda y: y if y is NOT a proper pseudolabel)
        #ds_l = ds_l.concatenate(newly_labeled)

        tf.print(full_x_u.shape, full_x_l.shape)
        y_dim = y_u.shape[0]
        tf.print(y_u.shape)
        tf.print("number unlabeled samlples {}".format(y_dim))
       
        tf.print(full_x_u.shape)
        new_x_l = [full_x_u[i, :, :, :] for i in range(y_dim) if y_u[i] > -1]
        new_y_l = [y_u[i] for i in range(y_dim) if y_u[i] > -1]

        if len(new_x_l) > 0:
          new_x_l = np.stack(new_x_l)
          new_y_l = np.stack(new_y_l)
          
          new_x_u = [full_x_u[i, :, :, :] for i in range(y_dim) if y_u[i] == -1]
          full_x_u = new_x_u

          if len(full_x_u) > 0:
            full_x_u = np.stack(full_x_u)
        
          full_x_l = np.concatenate((full_x_l, new_x_l))
          full_y_l = np.concatenate((full_y_l, new_y_l))


        labeled_loss.reset_states()
        unlabeled_loss.reset_states()
        accuracy.reset_states()
            

In [137]:

# hyperparams
lamda = 1     # proportion of unlabeled loss in total loss
eta = 0.03    # learning rate
beta = 0.09   # momentum
tau = 0.95    # threshold in pseudo-labeling
mu = 0.7      # proportion of unlabeled samples in batch
B = 64        # number of labeled examples in batch(in training)
K = 2 ** 20
nesterov = False
batch_size = 2  # should be 64?
# weight decay
# SGD instead of Adam


#CTAugment params
cta_classes = 10
cta_decay = 0.99
cta_depth = 2
cta_threshold = 0.8

hparams = {'lamda': lamda, 'eta': eta, 'beta': beta, 'tau': tau, 'mu': mu, 'B': B, 'K': K, 'nesterov': False, 'batch_size': batch_size,
           'cta_classes': cta_classes, 'cta_decay': cta_decay, 'cta_depth': cta_depth, 'cta_threshold': cta_threshold}

n_classes = 10

In [138]:
model = WRN_28_2()


In [139]:
def ParseFunction(serialized, image_shape=[32, 32, 3]):
    features = {'image': tf.io.FixedLenFeature([], tf.string),
                'label': tf.io.FixedLenFeature([], tf.int64)}

    parsed_example = tf.io.parse_single_example(serialized=serialized, features=features) 
    image = tf.image.decode_image(parsed_example['image'])
    image.set_shape(image_shape)
    # image = tf.cast(image, tf.float32) * (2.0 / 255) - 1.0
    data = dict(image=image, label=parsed_example['label'])
    return data

def stl_ParseFunction(input):
  return ParseFunction(serialized=input, image_shape=[96, 96, 3])

def LoadData(filename, tensor=False):
  dataset = tf.data.TFRecordDataset(filename)
  dataset = dataset.map(ParseFunction)
  
  # it = tf.compat.v1.data.make_one_shot_iterator(dataset) # Never used?
  images = np.stack([x['image'] for x in dataset])
  labels = np.stack([x['label'] for x in dataset])

  if tensor:
      return tf.data.Dataset.from_tensor_slices((images, labels))
  else:
      return images, labels

  def split_data_into_arrays(ds):
    images = []
    labels = []
    for ex in ds.take(50000000):
      try:
        images.append(ex[0])
        labels.append(ex[1])
      except:
        break
    return np.stack(images), np.stack(labels)


In [140]:
fpath = '/content/drive/My Drive/Colab Notebooks/cifar10.3@10-label.tfrecord'

ds_l = LoadData(fpath, tensor=True)

ds_u = LoadData(fpath, tensor=True)

In [141]:
training(model, ds_l, ds_u, hparams, n_classes)

epoch 1/5:   0%|                                                           | 0/2000 [00:00<?, ?it/s]

step
array([-1,  3])


epoch 1/5:   0%|                                                   | 1/2000 [00:00<12:41,  2.63it/s]

step
array([-1,  6])


epoch 1/5:   0%|                                                   | 2/2000 [00:00<10:39,  3.12it/s]

...

epoch 2/5:   0%|1                                                  | 4/2000 [00:00<05:31,  6.02it/s]

step
array([-1,  3])


epoch 2/5:   0%|1                                                  | 5/2000 [00:00<05:31,  6.02it/s]

(5, 32, 32, 3) (15, 32, 32, 3)
(10,)
number unlabeled samlples 10
(5, 32, 32, 3)


IndexError: ignored

In [ ]:
def test_error(model, test_data, test_labels):
  out = model(test_data)
  out_l = tf.math.argmax(out, axis=1)
  return np.sum(out_l == test_labels)/len(test_labels)

test_error(model, im, ls)

TypeError: ignored